<a href="https://colab.research.google.com/github/FunmilolaAdetuwo/Final-Project/blob/main/Advanced_AI_Techniques_for_Predicting_Currency_Exchange_Rates_A_Deep_Learning_Approach.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Instal the World bank library

!pip install wbdata

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 5.9 MB/s eta 0:00:00
  Attempting uninstall: tabulate
    Found existing installation: tabulate 0.9.0
    Uninstalling tabulate-0.9.0:
      Successfully uninstalled tabulate-0.9.0
  Attempting uninstall: decorator
    Found existing installation: decorator 4.4.2
    Uninstalling decorator-4.4.2:
      Successfully uninstalled decorator-4.4.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
bigframes 1.27.0 requires tabulate>=0.9, but you have tabulate 0.8.10 which is incompatible.
moviepy 1.0.3 requires decorator<5.0,>=4.0.2, but you have decorator 5.1.1 which is incompatible.


In [1]:
# Instal the keras tuner library
!pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.6 MB/s eta 0:00:00


In [2]:
# Importing necessary libraries

import yfinance as yf
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import wbdata as wb
import seaborn as sns
import matplotlib.dates as mdates
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input, BatchNormalization
from statsmodels.tsa.arima.model import ARIMA
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.layers import Conv1D, Dense, Flatten, Dropout
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
from tensorflow.keras.optimizers import Adam
from keras.layers import GRU, Dropout, Dense
from keras.models import Sequential
import kerastuner as kt
from kerastuner.tuners import RandomSearch
from tensorflow.keras.regularizers import l2

<ipython-input-2-31c1e52c3c7b>:23: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


First, we will be fetching the currency data from the Yahoo Finance. The GBP and the USD will be the currency that will be used for this project The period that will be used for this project will be from 01 January 2004 to 31 December 2023

In [3]:
# Defining the ticker symbol for GBP/USD
tcker_symb = "GBPUSD=X"

# Setting the data range
strt_dt = "2004-01-01"
ed_dt = "2023-12-31"

# Fetching the data
crcy_dt = yf.download(tcker_symb, start=strt_dt, end=ed_dt)

# setting the index to convert the index to a date column
crcy_dt.reset_index(inplace=True)

# Flattening the columns
crcy_dt.columns = ['_'.join(col).strip() if col[1] else col[0] for col in crcy_dt.columns.values]

crcy_dt['Date'] = pd.to_datetime(crcy_dt['Date'])


# Display the data structure after modification
print("Data after flattening columns and setting 'Date':")
print(crcy_dt.head())

[*********************100%***********************]  1 of 1 completed

Data after flattening columns and setting 'Date':
        Date  Adj Close_GBPUSD=X  Close_GBPUSD=X  High_GBPUSD=X  Low_GBPUSD=X  \
0 2004-01-01            1.785204        1.785204       1.788813      1.781705   
1 2004-01-02            1.793690        1.793690       1.795010      1.779201   
2 2004-01-05            1.807207        1.807207       1.810315      1.795107   
3 2004-01-06            1.821295        1.821295       1.827786      1.805510   
4 2004-01-07            1.817785        1.817785       1.825584      1.812711   

   Open_GBPUSD=X  Volume_GBPUSD=X  
0       1.788813                0  
1       1.785587                0  
2       1.795493                0  
3       1.807403                0  
4       1.820897                0  


Now, using the same period, we will fetch the macro economic data from the World bank for the United Kingdom and United States.

The indicators to be used is the GDP and Inflation rate.

In [4]:
# Setting the date range
dt_dte = ("2004", "2023")

# Getting the GDP and Inflation data for US and UK
gdp_dt = wb.get_data("NY.GDP.MKTP.CD", country=['GBR', 'USA'], date=dt_dte)
ifltn_dt = wb.get_data("FP.CPI.TOTL.ZG", country=['GBR', 'USA'], date=dt_dte)

# Converting this into DataFrames
gdp_df = pd.DataFrame(gdp_dt)
ifltn_df = pd.DataFrame(ifltn_dt)

def extract_dict_value(row, key):
    return row[key]['value'] if isinstance(row[key], dict) else row[key]

gdp_df['country'] = gdp_df['country'].apply(extract_dict_value, key='value')
gdp_df['indicator'] = gdp_df['indicator'].apply(extract_dict_value, key='value')

ifltn_df['country'] = ifltn_df['country'].apply(extract_dict_value, key='value')
ifltn_df['indicator'] = ifltn_df['indicator'].apply(extract_dict_value, key='value')

gdp_df['date'] = pd.to_datetime(gdp_df['date'])
ifltn_df['date'] = pd.to_datetime(ifltn_df['date'])

gdp_pt = gdp_df.pivot(index='date', columns='country', values='value').rename(columns=lambda x: f"GDP_{x}")
ifltn_pt = ifltn_df.pivot(index='date', columns='country', values='value').rename(columns=lambda x: f"Inflation_{x}")

# Merging the GDP and Inflation data into a single DataFrame
ecomc_dt = gdp_pt.join(ifltn_pt, how='outer')

# Displaying the data
print(ecomc_dt.head())

country     GDP_United Kingdom  GDP_United States  Inflation_United Kingdom  \
date                                                                          
2004-01-01        2.421525e+12       1.221720e+13                  1.390398   
2005-01-01        2.543180e+12       1.303920e+13                  2.089136   
2006-01-01        2.708442e+12       1.381558e+13                  2.455662   
2007-01-01        3.090510e+12       1.447423e+13                  2.386562   
2008-01-01        2.929412e+12       1.476986e+13                  3.521409   

country     Inflation_United States  
date                                 
2004-01-01                 2.677237  
2005-01-01                 3.392747  
2006-01-01                 3.225944  
2007-01-01                 2.852672  
2008-01-01                 3.839100  


The Currency data and the Macroeconomic data is merged together. The daily exchange rate data of GBP/USD with annual GDP and inflation data for the UK and the US was merged together. In this integration, there is high-frequency daily data, which shows trends in currency exchange rate, with lower-frequency annual data that shows the macro economic data.

The datasets are different in frequency—daily for exchange rates and annually for macro economic indicators. This was aligned by using a forward fill method, which carries forward the last known annual values until new data becomes available. This means every daily exchange rate from January 1, 2004, to December 29, 2023, is associated with the latest annual macro economic data.

In [5]:
crcy_dt['Date'] = crcy_dt['Date'].dt.tz_localize(None)
ecomc_dt.index = ecomc_dt.index.tz_localize(None)

# Merging the daily currency data with the economic data using forward fill to handle the missing daily values
cmbd_dt = pd.merge(crcy_dt, ecomc_dt, left_on='Date', right_index=True, how='left')
cmbd_dt.fillna(method='ffill', inplace=True)

# Displaying the combined data
print(cmbd_dt.head())

        Date  Adj Close_GBPUSD=X  Close_GBPUSD=X  High_GBPUSD=X  Low_GBPUSD=X  \
0 2004-01-01            1.785204        1.785204       1.788813      1.781705   
1 2004-01-02            1.793690        1.793690       1.795010      1.779201   
2 2004-01-05            1.807207        1.807207       1.810315      1.795107   
3 2004-01-06            1.821295        1.821295       1.827786      1.805510   
4 2004-01-07            1.817785        1.817785       1.825584      1.812711   

   Open_GBPUSD=X  Volume_GBPUSD=X  GDP_United Kingdom  GDP_United States  \
0       1.788813                0        2.421525e+12       1.221720e+13   
1       1.785587                0        2.421525e+12       1.221720e+13   
2       1.795493                0        2.421525e+12       1.221720e+13   
3       1.807403                0        2.421525e+12       1.221720e+13   
4       1.820897                0        2.421525e+12       1.221720e+13   

   Inflation_United Kingdom  Inflation_United States  
0

<ipython-input-5-c6f190ef9204>:6: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  cmbd_dt.fillna(method='ffill', inplace=True)
